In [ ]:
# Third test for git aliases (e.g., git-update-plus, git-next-checkin, etc)
#
# Note: The aliases should be defined before Jupyter is invoked.
# For BatsPP, they can loaded via '--source all-tomohara-aliases-etc.bash' option.

In [ ]:
# Global Setup
# creating temp local repos to test on
TMPA=/tmp/temp-git-test

In [ ]:
# Global setup
# Define aliases for encoding output (e.g., to ensure output the same across runs).
## OLD: alias encode-control-chars='perl -pe "s/\t/<tab>/g;  s/\n/<nl>/g;  s/[\x00-\x31]/<cc>/g;"'
alias encode-control-chars='perl -pe "s/\t/<tab>/g;  s/[\x00-\x08\x11\x31]/<cc>/g;"'
alias encode-nums='perl -pe "s/[0-9]+/N/g;"'
## TODO: define a better hex encoder to avoid false positives (e.g., abc).
alias encode-hex='perl -pe "s/[0-9A-F]{4,}/<hex>/ig;"'
# note: uses perl slurp mode (i.e., read entire file) to check for consecutive newlines
alias encode-newlines='perl -0777 -pe "while (/\n\n/) { s/\n\n/\n<newline>\n/g; }"'
function encode-text () { encode-control-chars | encode-nums | encode-hex | encode-newlines; }

In [ ]:
# Global Setup
git clone https://github.com/tomasohara/git-bash-test.git "$TMPA" > /dev/null 2>&1

In [ ]:
# Setup
# Move to second local to push some changes
command cd "$TMPA" > /dev/null 2>&1
git pull origin main > /dev/null 2>&1
echo "hello second local" > hello.txt 
git add hello.txt > /dev/null 2>&1
git commit -m "first commit on second local" > /dev/null 2>&1
git push > /dev/null 2>&1

In [ ]:
# Setup
# git-diff-plus tests
cat hello.txt > bye.txt
echo "differ" >> hello.txt
diffs=$(git-diff-plus hello.txt bye.txt | grep "^\+\w" | wc -l)

In [ ]:
# Assert 
# there should only be one line of difference
[ $diffs -eq 1 ]; echo $?;

In [ ]:
# Setup
# git-diff-list tests
changed=$(git-diff-list | wc -l)

In [ ]:
# Assert 
# there should be one file changed
[ $changed -eq 1 ]; echo $?;

In [ ]:
# Setup
# git-vdiff-alias tests
killall nvim > /dev/null 2>&1
git-vdiff-alias > /dev/null 2>&1
instances=$(ps -e | grep "nvim" | wc -l)

In [ ]:
# Assert there is an instance of nvim 
[ $instances -eq 1 ]; echo $?;

In [ ]:
# Setup
# git-next-checkin tests
rm -f bye.txt > /dev/null 2>&1
previous_changes=$( git status -s | wc -l )

In [ ]:
# Setup
# set env variables and run git-next-checkin
export GIT_MESSAGE="checkin"
previous=$GIT_NO_CONFIRM
export GIT_NO_CONFIRM="1"
git-next-checkin hello.txt > /dev/null 2>&1

In [ ]:
# Setup
# delete new files created by git-next-cheking and count changes
rm -rf $(ls | grep '[.][0-9]*[.]') > /dev/null 2>&1
changes=$( git status -s | wc -l )

In [ ]:
# Assert there were changes before, and no changes now
[ $previous_changes -gt 1 ] && [ $changes -eq 0 ]; echo $?;

In [ ]:
# Setup
# undo changes and remove newly created files
# TODO2:
## also, converting unique ID's to codes like temp-git-test/.git/objects/pack/pack-HEX.idx
## EX: cat $log | perl -pe 's/\b[0-9]+\b/NUM/g; s/\b[a-f0-9]+\b/HEX/g;'
rm -f *.txt > /dev/null 2>&1
rm -rf $(ls | grep '[.][0-9]*[.]') > /dev/null 2>&1
git rm -f *.txt > /dev/null 2>&1
git add . > /dev/null 2>&1

In [ ]:
# Setup
## Lorenzo-TODO2: use one cell per output (to make this less brittle)
git commit -m "cleanup test files" > /dev/null 2>&1
git push > /dev/null 2>&1

In [ ]:
# Setup
rename-with-file-date /tmp/temp-git-test > /dev/null 2>&1
export GIT_NO_CONFIRM=$previous